In [54]:
import os
from pathlib2 import Path
import re
import itertools
import copy
import numpy as np

import pyarrow.feather as feather
import pandas as pd
import xml.etree.ElementTree as ET
from dython.nominal import associations
from xgboost import XGBRegressor

In [3]:
category_data = feather.read_feather(Path('IceCat_Cat_2833_feather/frame_IceCat_Category_909.feather'))
display(category_data.tail())
category_data.info()

,level_0,index,id,name,category_id,category_label,Operating temperature (T-T).1112,Operating temperature (T-T).1112.unit,Maximum data transfer rate.1165,Maximum data transfer rate.1165.unit,...,Compatibility.890,Compatibility.890.unit,Interface type.990,Interface type.990.unit,Firewall security.1612,Firewall security.1612.unit,Works with the Google Assistant.36516,Works with the Google Assistant.36516.unit,RTS/CTS threshold.22398,RTS/CTS threshold.22398.unit
2232,2232,0,IceCat_Prod_16048998,Aironet 2602E,IceCat_Category_909,Wireless Access Points,-20 - 55,°C,450.0,Mbit/s,...,None,None,None,None,None,None,<NA>,None,<NA>,None
2233,2233,0,IceCat_Prod_31985083,Aironet 3700i,IceCat_Category_909,Wireless Access Points,0 - 40,°C,1300.0,Mbit/s,...,None,None,None,None,None,None,<NA>,None,<NA>,None
2234,2234,0,IceCat_Prod_33721921,Aironet 3600e,IceCat_Category_909,Wireless Access Points,-20 - 55,°C,1000.0,Mbit/s,...,None,None,None,None,None,None,<NA>,None,<NA>,None
2235,2235,0,IceCat_Prod_11147703,NWA5550-N,IceCat_Category_909,Wireless Access Points,-40 - 60,°C,300.0,Mbit/s,...,None,None,None,None,None,None,<NA>,None,<NA>,None
2236,2236,0,IceCat_Prod_36197901,Aironet 1562E,IceCat_Category_909,Wireless Access Points,-40 - 65,°C,1300.0,Mbit/s,...,None,None,None,None,None,None,<NA>,None,<NA>,None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2237 entries, 0 to 2236
Columns: 540 entries, level_0 to RTS/CTS threshold.22398.unit
dtypes: boolean(88), float64(69), int64(2), object(381)
memory usage: 8.1+ MB


In [4]:
pct_complete = (len(category_data) - category_data.isnull().sum()) / len(category_data)
missing_vals = pd.DataFrame({'col': category_data.columns,
                             'pct_complete': pct_complete})
# missing_vals[missing_vals['pct_complete'] >= .75]


In [5]:
cols_of_interest = missing_vals[missing_vals['pct_complete'] >= .75].iloc[:,0]
category_data[category_data.columns.intersection(cols_of_interest)]

,level_0,index,id,name,category_id,category_label,Operating temperature (T-T).1112,Operating temperature (T-T).1112.unit,Maximum data transfer rate.1165,Maximum data transfer rate.1165.unit,...,Ethernet LAN (RJ-45) ports.2312,Ethernet LAN data rates.3768,Security algorithms.454,Storage temperature (T-T).757,Storage temperature (T-T).757.unit,Weight.94,Weight.94.unit,LED indicators.1501,Operating relative humidity (H-H).703,Operating relative humidity (H-H).703.unit
0,0,0,IceCat_Prod_26835138,Aironet 1550,IceCat_Category_909,Wireless Access Points,-40 - 55,°C,1000.0,Mbit/s,...,1.0,"10,100,1000","802.1x RADIUS,EAP,EAP-TLS,EAP-TTLS,LEAP,PEAP,W...",-50 - 85,°C,8000.0,g,None,None,None
1,1,0,IceCat_Prod_33721842,Aironet 1550,IceCat_Category_909,Wireless Access Points,-40 - 55,°C,1000.0,Mbit/s,...,NaN,"10,100,1000","802.1x RADIUS,EAP,EAP-TLS,EAP-TTLS,LEAP,PEAP,W...",-50 - 85,°C,7800.0,g,None,None,None
2,2,0,IceCat_Prod_93242097,NWA1123ACv3,IceCat_Category_909,Wireless Access Points,0 - 50,°C,866.0,Mbit/s,...,1.0,None,"802.1x RADIUS,EAP,WEP,WPA,WPA2-Enterprise,WPA2...",-40 - 70,°C,249.0,g,Y,10 - 90,%
3,3,0,IceCat_Prod_85859936,C9105AXW-Q,IceCat_Category_909,Wireless Access Points,0 - 50,°C,NaN,None,...,1.0,"10,100,1000",None,-30 - 70,°C,373.2,g,None,10 - 90,%
4,4,0,IceCat_Prod_29879834,Aironet 1572EAC,IceCat_Category_909,Wireless Access Points,-40 - 65,°C,1300.0,Mbit/s,...,1.0,"10,100,1000",None,-50 - 70,°C,6100.0,g,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,2232,0,IceCat_Prod_16048998,Aironet 2602E,IceCat_Category_909,Wireless Access Points,-20 - 55,°C,450.0,Mbit/s,...,1.0,"10,100,1000","EAP,EAP-TLS,EAP-TTLS,PEAP,TKIP,WPA,WPA2",-30 - 70,°C,1224.0,g,Y,10 - 90,%
2233,2233,0,IceCat_Prod_31985083,Aironet 3700i,IceCat_Category_909,Wireless Access Points,0 - 40,°C,1300.0,Mbit/s,...,2.0,"10,100,1000","802.1x RADIUS,AES,EAP-FAST,EAP-PEAP,EAP-SIM,EA...",-30 - 70,°C,1130.0,g,Status,10 - 90,%
2234,2234,0,IceCat_Prod_33721921,Aironet 3600e,IceCat_Category_909,Wireless Access Points,-20 - 55,°C,1000.0,Mbit/s,...,1.0,"10,100,1000","AES,EAP-FAST,EAP-PEAP,EAP-SIM,EAP-TLS,EAP-TTLS...",-30 - 70,°C,1130.0,g,Y,10 - 90,%
2235,2235,0,IceCat_Prod_11147703,NWA5550-N,IceCat_Category_909,Wireless Access Points,-40 - 60,°C,300.0,Mbit/s,...,1.0,"10,100,1000","WEP,WPA,WPA2,WPA2-PSK",-40 - 70,°C,1360.0,g,Y,10 - 90,%


In [34]:
def remove_data(df, p_rm, id_cols = []):
    df_copy = copy.deepcopy(df)

    for col in df_copy:
        if col in id_cols:
            continue
        n_val = df_copy[col].notna().sum()
        n_rm = int(n_val * p_rm)
        idx_ = np.random.choice(df_copy[col].shape[0], n_rm, replace=False)

        df_copy.loc[idx_, col] = np.nan

    return df_copy 

In [37]:
category_data_xgboost = category_data[["name", "Maximum data transfer rate.1165", "Product colour.1766"]]

category_data_xgboost_prod_col_rm = remove_data(category_data_xgboost, 0.3, ["name", "Maximum data transfer rate.1165"])
category_data_xgboost_col_nan_idx = category_data_xgboost_prod_col_rm[category_data_xgboost_prod_col_rm["Product colour.1766"].isnull()].index
category_data_xgboost_train = category_data_xgboost_prod_col_rm.drop(category_data_xgboost_col_nan_idx, axis=0)
category_data_xgboost_test = category_data_xgboost_prod_col_rm[category_data_xgboost_prod_col_rm.index.isin(category_data_xgboost_col_nan_idx)]

display(category_data_xgboost_train)
display(category_data_xgboost_test)

,name,Maximum data transfer rate.1165,Product colour.1766
1,Aironet 1550,1000.0,White
2,NWA1123ACv3,866.0,White
3,C9105AXW-Q,NaN,Grey
5,WIRELESS ACCESS POINT,1300.0,Grey
6,Aironet 702i,1000.0,White
...,...,...,...
2230,Aironet 1700,1000.0,White
2231,WAP125,867.0,White
2233,Aironet 3700i,1300.0,White
2234,Aironet 3600e,1000.0,White


,name,Maximum data transfer rate.1165,Product colour.1766
0,Aironet 1550,1000.0,NaN
4,Aironet 1572EAC,1300.0,NaN
7,Aironet 2802i,5200.0,NaN
8,Aironet 3502I,300.0,NaN
9,WNDAP360,1000.0,None
...,...,...,...
2227,Aironet 1600,300.0,NaN
2228,Aironet 3802i,5200.0,NaN
2229,WNAP210,100.0,None
2232,Aironet 2602E,450.0,NaN


In [51]:
def factorize(df):
    df_copy = df.copy()
    for col in df_copy:
        unique_items = df[col].dropna().unique().tolist()
        mapping_dict = {k:v for v, k in enumerate(unique_items)}
        df_copy[col] = df_copy[col].map(mapping_dict)
    return df_copy

train_factorized = factorize(category_data_xgboost_train)
test_factorized = factorize(category_data_xgboost_test)

In [75]:
test_factorized["name"][4]

1

In [79]:
model = XGBRegressor()
model.fit(train_factorized.drop("Product colour.1766", axis=1), train_factorized["Product colour.1766"])
pred_col = model.predict(test_factorized.drop("Product colour.1766", axis=1))
pred_col.astype(int)

unique_items = category_data_xgboost_train["Product colour.1766"].dropna().unique().tolist()
mapping_dict = {k:v for v, k in enumerate(unique_items)}
encoding_map = {v:k for k,v in mapping_dict.items()}

category_data_xgboost_prod_col_rm_copy = category_data_xgboost_prod_col_rm.copy()
category_data_xgboost_prod_col_rm_copy["Product colour.1766"][category_data_xgboost_col_nan_idx] = pd.Series(pred_col.astype(int)).map(encoding_map)


C:\Users\BensonIgarabuza\AppData\Local\Temp\ipykernel_36740\2595731707.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data_xgboost_prod_col_rm_copy["Product colour.1766"][category_data_xgboost_col_nan_idx] = pd.Series(pred_col.astype(int)).map(encoding_map)


In [93]:
display(category_data_xgboost_prod_col_rm_copy[category_data_xgboost_prod_col_rm_copy.index.isin(category_data_xgboost_col_nan_idx)])
display(category_data_xgboost[category_data_xgboost.index.isin(category_data_xgboost_col_nan_idx)])

merged = category_data_xgboost_prod_col_rm_copy[category_data_xgboost_prod_col_rm_copy.index.isin(category_data_xgboost_col_nan_idx)].join(category_data_xgboost[category_data_xgboost.index.isin(category_data_xgboost_col_nan_idx)], how='inner', rsuffix='t2')
merged["acc"] = merged["Product colour.1766"] == merged["Product colour.1766t2"]
merged["acc"] = merged["acc"].apply(lambda x: int(x))
merged["acc"].sum()

,name,Maximum data transfer rate.1165,Product colour.1766
0,Aironet 1550,1000.0,White
4,Aironet 1572EAC,1300.0,White
7,Aironet 2802i,5200.0,White
8,Aironet 3502I,300.0,White
9,WNDAP360,1000.0,White
...,...,...,...
2227,Aironet 1600,300.0,White
2228,Aironet 3802i,5200.0,White
2229,WNAP210,100.0,White
2232,Aironet 2602E,450.0,White


,name,Maximum data transfer rate.1165,Product colour.1766
0,Aironet 1550,1000.0,White
4,Aironet 1572EAC,1300.0,Grey
7,Aironet 2802i,5200.0,White
8,Aironet 3502I,300.0,None
9,WNDAP360,1000.0,None
...,...,...,...
2227,Aironet 1600,300.0,Grey
2228,Aironet 3802i,5200.0,White
2229,WNAP210,100.0,None
2232,Aironet 2602E,450.0,None


368

In [51]:
tree = ET.parse('IceCat Specifications/CategoriesList.xml')
root = tree.getroot()

In [52]:
# Get all categories from specification
category_parent_list = []
for category in root[0][0].findall('Category'):
    cat_id = category.attrib
    parent_id = category.find('ParentCategory').attrib
    category_parent_list.append((int(cat_id["ID"]), int(parent_id["ID"])))

# Construct tree of categories from specification
all_nodes = {n[0]: {} for n in category_parent_list}
tree = {}
for item in category_parent_list:
    id, parent = item
    if parent != 1:
        all_nodes[parent][id] = all_nodes[id]
    else:
        tree[id] = all_nodes[id]




In [53]:
# Get categories from provided feather files
feather_file_names = os.listdir("IceCat_Cat_2833_feather")
feather_categories = [int(re.search(r'[0-9]+', file_name).group(0)) for file_name in feather_file_names]


def dict_key_filter(obj, obj_filter):
    '''
    Filters dictionary to only include `obj` keys that are included in `obj_filter`. 
    https://stackoverflow.com/questions/31710271/how-to-filter-by-keys-through-a-nested-dictionary-in-a-pythonic-way
    '''
    def inner_dict_key_filter(obj): return dict_key_filter(obj, obj_filter)
    def to_keep(subtree): return not isinstance(subtree, (dict, list)) or subtree

    def build_subtree(key, value):
        if key in obj_filter:
            return copy.deepcopy(value) # keep the branch
        elif isinstance(value, (dict, list)):
            return inner_dict_key_filter(value) # continue to search
        return [] # just an orphan value here

    if isinstance(obj, dict):
        key_subtree_pairs = ((key, build_subtree(key, value)) for key, value in obj.items())
        return {key:subtree for key, subtree in key_subtree_pairs if to_keep(subtree)}
    elif isinstance(obj, list):
        return list(filter(to_keep, map(inner_dict_key_filter, obj)))
    return []


filtered_tree = dict_key_filter(tree, feather_categories)




In [1]:
filtered_tree

NameError: name 'filtered_tree' is not defined

In [60]:

def flatten(d):    
    res = []
    if isinstance(d, dict):
        for key, val in d.items():
            res.append(key)
            res.extend(flatten(val))
    return res

collapsed_cat_filtered_tree = flatten(filtered_tree)
len(collapsed_cat_filtered_tree)

562

In [84]:
filtered_tree[2833][2][55][60]

{}

In [85]:
cpy = copy.copy(collapsed_cat_filtered_tree)
cpy.sort()
len(cpy)

562

In [92]:
level_one_count = 0
level_two_count = 0
level_three_count = 0
level_four_count = 0
level_one_items  = []
level_two_items = []
level_three_items = []
level_four_items = []
for level_one in filtered_tree.keys():
    level_one_count += 1
    level_one_items.append(level_one)
    for level_two in filtered_tree[level_one].keys():
        level_two_count += 1
        level_two_items.append(level_two)
        for level_three in filtered_tree[level_one][level_two].keys():
            level_three_items.append(level_three)
            level_three_count += 1
            for level_four in filtered_tree[level_one][level_two][level_three].keys():
                level_four_count += 1
                level_four_items.append(level_four)



In [88]:
[level_one_count, level_two_count, level_three_count, level_four_count]

[1, 11, 212, 338]

In [91]:
level_three_items

[7,
 12,
 21,
 26,
 27,
 28,
 38,
 42,
 55,
 62,
 67,
 803,
 836,
 885,
 893,
 904,
 1534,
 1535,
 1552,
 1624,
 1646,
 1657,
 1719,
 1753,
 2444,
 2841,
 2842,
 2843,
 3912,
 5210,
 5230,
 5231,
 5640,
 7100,
 7241,
 9225,
 166,
 178,
 189,
 236,
 998,
 1291,
 1762,
 2702,
 2804,
 2844,
 2901,
 5984,
 6892,
 8839,
 8917,
 151,
 153,
 154,
 155,
 156,
 159,
 203,
 896,
 897,
 1062,
 1389,
 1508,
 1573,
 2093,
 2282,
 2774,
 2775,
 3319,
 3625,
 5022,
 5400,
 5459,
 5467,
 5632,
 5831,
 6946,
 7284,
 7291,
 7319,
 7396,
 8179,
 8189,
 8194,
 8231,
 8355,
 8738,
 8909,
 8910,
 8919,
 8921,
 9228,
 192,
 194,
 195,
 196,
 198,
 202,
 204,
 282,
 1041,
 1297,
 1540,
 2675,
 2813,
 2929,
 3955,
 8154,
 8470,
 8500,
 8603,
 8685,
 287,
 2840,
 8763,
 8764,
 373,
 692,
 1305,
 4972,
 4973,
 8653,
 2503,
 8752,
 8753,
 8754,
 8755,
 8756,
 8757,
 8758,
 867,
 869,
 883,
 952,
 953,
 956,
 1121,
 1158,
 1303,
 1306,
 1493,
 1494,
 1495,
 1501,
 1569,
 1574,
 1582,
 1614,
 1638,
 1639,
 1663,
 1

In [46]:
def flatten(d):    
    res = []  # Result list
    if isinstance(d, dict):
        for key, val in d.items():
            res.append(key)
            res.extend(flatten(val))
    # elif isinstance(d, list):
    #     res = d        
    # else:
    #     raise TypeError("Undefined type for flatten: %s"%type(d))

    return res


dict1 = {
    'Bob': {
        'shepherd': [4, 6, 3],
        'collie': [23, 3, 45],
        'poodle': [2, 0, 6],
    },
    'Sarah': {
        'shepherd': [1, 2, 3],
        'collie': [3, 31, 4],
        'poodle': [21, 5, 6],
    },
    'Ann': {
        'shepherd': [4, 6, 3],
        'collie': [23, 3, 45],
        'poodle': [2, 10, 8],
    }
}

print( flatten(dict1) )

['Bob', 'shepherd', 'collie', 'poodle', 'Sarah', 'shepherd', 'collie', 'poodle', 'Ann', 'shepherd', 'collie', 'poodle']


In [19]:
level_two_items

[2, 106, 150, 191, 206, 220, 225, 242, 830, 839, 2557]

In [13]:
len(res[2833])


NameError: name 'res' is not defined

In [ ]:
st ='frame_IceCat_Category_1007.feather'
re.search(r'[0-9]+', st).group(0)

'1007'